# Preprocessing

Die Dateien, die wir runtergeladen haben. Waren ~1300 Chunks mit jeweils 1000 Tweets, die aber nicht alle von den Politikern selbst gepostet wurden, sondern teilweise auch von deren Anhängern kommentiert oder geretweeted wurden. Das heißt wir mussten erst einmal aussortieren um an die richtigen Tweets zu kommen.
Anschließend haben wir den Text bereinigt und tokenisiert.

In [1]:
# Importiere die benötigten packages
import pandas as pd
import json
import numpy as np
import os
import re

## Hilfsfunktionen

In [2]:
''' Diese Funktion geht durch alle Elemente von "all_tweets" und speichert nur die ab, die bei einem Mitglied der Partei ist
    Input:  Party = Liste der Screen_names einer politischen Partei
            all_tweets = eine Liste der originalen .json Dateien
    Output: eine Liste der Tweets, die von einem der Mitglieder der Partei gepostet wurden
'''
def get_tweets_by(party, all_tweets):
    # erstelle eine leere Liste um die Tweets abzuspeichern
    tweets = []
    # iteriere über alle Tweets in der Datei
    for i in range(len(all_tweets)):
        # iteriere über alle Mitglieder der Partei
        for member in party:
            # falls der aktuelle Tweets von dem aktuellen Mitglied ist: hänge diesen Tweet der Liste an
            if (all_tweets[i]['user']['screen_name'] == member):
                tweets.append(all_tweets[i]['text'])
    return tweets

''' Diese Funktion entfernt alle Zeichen, die wir nicht haben wollen aus der Textdatei. Da die Tweets zunächst als Listenelemente
    aneinander gereit wurden (siehe Preprocessing), sind in der Textdatei noch sehr viele eckige Klammern. Außerdem wurden 
    Zeilenumbrüche als '/n' codiert und Tweets, die die Standardlänge von 140 Zeichen überschritten haben, wurden abgeschnitten,
    sodass teilweise unvollständige Worte vorkommen. Zudem gibt es noch einige weitere ungewünschte Zeichen, wie URL-links etc.
    Input:  Eine String Datei
    Output: Die Datei ohne die unerwünschten Zeichen
'''
def clean_text(text):
    
    # entfernt URL Links
    text = re.sub(r"http\S+", "", text)
    # entfernt @screen_name
    text = re.sub(r"@\S+", "", text)
    # entfernt "RT" für Re-Tweet
    text = re.sub(r"RT", "", text)
    # entfernt Sondersymbole wie Emojis
    text = re.sub(r"🇩🇪", "", text)
    #text = re.sub(r"😁", "", text)
    #text = re.sub(r"😀", "", text)
    #text = re.sub(r"'❤'", "", text)
    #text = re.sub(r"❤", "", text)
    #text = re.sub(r"🎉", "", text)
    # entfernt Sonderzeichen
    text = re.sub(r"'️", "", text)
    text = re.sub(r"…", "", text)
    text = re.sub(r"\\n", "", text)
    text = re.sub(r"\'", "", text)
    text = re.sub(r"\\", "", text)
    text = re.sub(r"\n", "", text)
    text = re.sub(r"]\S+", "", text)
    # entfernt einzelne Symbole 
    text = ''.join( c for c in text if  c not in '[,],/,:,&,_,1,2,3,4,5,6,7,8,9,0,,' )

    # ersetzt Umlaute und ß
    text = re.sub(r"ü", "ue", text)
    text = re.sub(r"ö", "oe", text)
    text = re.sub(r"ä", "ae", text)
    text = re.sub(r"ß", "ss", text)
    
    # setzt ein Leerzeichen vor jeden Punkt, sodass er als einzelnes Symbol gesehen wird und Wörter mit Punkt
    # nicht als eigenständige Wörter ins Vokabular eingehen
    text = re.sub(r"\.", " .", text)
    
    return text

## Einlesen der Daten

In [3]:
# Teil des Datensets, welches wir runtergeladen haben, war auch eine Liste aller Twitter Accounts von Politikern
# sortiert nach der Parteizugehörigeit.
# Die Listen befinden sich in der Datei followed-accounts, die zunächst eingelesen wird
with open('followed-accounts.json') as json_file:
    followed_accounts = json.load(json_file)

# Jetzt müssen die Namen der einzelnen Accounts nur noch getrennt abgespeichert werden
CDU = followed_accounts['CDU/CSU']
SPD = followed_accounts['SPD']
FDP = followed_accounts['FDP']
LINKE = followed_accounts['Linke']
GRUENE = followed_accounts['Grüne']
AFD = followed_accounts['AfD']


In [ ]:
# "path" gibt an, wo die Twitterdaten gespeichert sind
path = 'recorded-tweets/chunk1'
# in "all_files" werden alle Verzeichnisse der einzelnen Chunks gespeichert
all_files = [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.json')]

# nun wird durch diese Verzeichnisse iteriert und die Chunks werden in eine einzelne Datei aneinander gehängt
# da es zu viele Dateien sind um sie alle auf einmal zu laden ohne unsere Laptops zu überfordern, haben wir sie in einzelnen
# Teilen geladen und weiter verarbeitet
data1 = []
for i in range(100):
    file = pd.read_json(all_files[i], orient='index')
    data1.append(file)


## Weiterverarbeitung der Tweets

In [54]:
# Für unser Model wollen wir nur die Tweets benutzen, die tatsächlich von einem Mitglied der jeweiligen Partei gepostet wurden.
# Daher sortieren wir alle anderen Tweets aus und speichern nur die für uns relevanten 
tweets_afd1 = []

for i in range(len(data1)):
    tweets_afd1.append(get_tweets_by(AFD, data1[i]))


35

In [77]:
# Aus der Liste mit allen Tweets wird nun ein String gemacht
tweets_afd1_str = ''.join(map(str,tweets_afd1))
# Der String wird mit der oben definierten Hilfsfunktion von ungewollten Symbolen befreit
clean_afd1 = clean_text(tweets_afd1_str)

## Abspeichern der fertigen Textdatei

In [82]:
f= open("tweets_afd1.txt","w+",encoding="utf8", errors='ignore')     
f.write(clean_afd1)
f.close()
